<a href="https://colab.research.google.com/github/iypc-team/CoLab/blob/master/Writefile_GetTPU_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import absolute_import
import os, shutil
import tensorflow as tf

try: shutil.rmtree('/content/sample_data')
except: pass
print('Updated: 10/18/2022-3')

In [ ]:
%%writefile GetTPU2.py
# Updated: 10/18/2022
# fall back to 10/16/2022

from __future__ import absolute_import
from google.colab import drive, files
import glob, os, re, time, shutil
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
from os.path import abspath, basename, exists, join
from pathlib import Path

import tensorflow as tf
from tensorflow.python.profiler import profiler_client
# from tensorflow.python.platform.tf_logging import log
import numpy as np
np.set_printoptions(4)
from matplotlib import pyplot as plt

contentPath=os.getcwd()

try: shutil.rmtree('/content/sample_data')
except: pass

if not exists('/content/drive'):
    drive.mount('/content/drive', force_remount=True)

shutil.copy2('/content/drive/MyDrive/BashColors.py', contentPath)
from BashColors import C

AUTO = tf.data.experimental.AUTOTUNE

class GetTPU(object):
    """with strategy.scope():"""
    gtp = None
    tpu = None
    strategy = None

    def __init__(self):
        os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
        self.updated='Updated: 10/18/2022'
        self.AUTO = tf.data.experimental.AUTOTUNE
        self.gtp = None
        self.tpu = None
        self.strategy = None

        # gtp = GetTPU
        self.connectTPU()
        tpu = self.getTPU()
        strategy = self.getStrategy()

        super(object, self).__init__()

    def __all__(self):
        return self.getMethodList()
    def __iter__(self):
        return self
    def __len__(self):
        return len(self.name)
    def __str__(self):
        return "%s(%r)" % (self.__class__, self.__dict__)

    def getMethodList(self, silent=True):
        '''List all methods in AllInstall\n Print silent = True'''
        method_list=[]
        for attribute in dir(self):
            # Get the attribute value
            attribute_value = getattr(self, attribute)
            # Check that it is callable
            if callable(attribute_value):
                # Filter all dunder (__ prefix) methods
                if attribute.startswith('__') == False:
                    method_list.append(attribute)
        if not silent:
            print(f'{len(method_list)} callable methods in AllInstall')
            for method in method_list:
                print(method)
        return method_list

    def getTPUInfo(self):
        """ """
        if tf.config.list_physical_devices('TPU'):
            from tensorflow.python.profiler import profiler_client
            tpu_profile_service_address = os.environ['COLAB_TPU_ADDR'].replace('8470', '8466')
            print(profiler_client.monitor(tpu_profile_service_address, 100, 2))
            tpu_profile_service_address = os.environ['COLAB_TPU_ADDR'].replace('8466', '8470')

    def getTPU(self):
        """ """
        return self.tpu

    def getStrategy(self):
        """ """
        return self.strategy

    def connectTPU(self):
        """ """
        # TPUClusterResolver automatically checks connected TPU on all Google platforms
        if tf.config.list_physical_devices('TPU'):
            self.tpu = None
            self.tpu = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection
            tf.config.experimental_connect_to_cluster(self.tpu)
            tf.tpu.experimental.initialize_tpu_system(self.tpu)
            self.strategy = tf.distribute.TPUStrategy(self.tpu)
            with self.strategy.scope():
                print(f'{C.BIWhite}Number of TPUs: {self.strategy.num_replicas_in_sync}{C.ColorOff}')
                # print('using TPU')
            # print(f'{C.BIRed}{err}{C.ColorOff}')

        elif tf.config.list_physical_devices('GPU'):
            self.tpu = tf.distribute.MirroredStrategy() # for GPU or multi-GPU machines
            # print(type(strategy))
            # with self.strategy.scope():
            print(f'{C.BIPurple}using GPU{C.ColorOff}')

        elif tf.config.list_physical_devices('CPU'):
            self.tpu = tf.distribute.get_strategy()# default strategy for CPU
            # with self.strategy.scope():
            print(f'{C.BIRed}using CPU{C.ColorOff}')
            
            # for clusters of multi-GPU machines
            # self.strategy=tf.distribute.experimental.MultiWorkerMirroredStrategy()

gtp = GetTPU()
# gtp.connectTPU()
# tpu = gtp.getTPU()
strategy = gtp.getStrategy()
# print(f'{C.BIWhite}{gtp.updated}{C.ColorOff}')

In [ ]:
from GetTPU2 import *
# print(gtp.__all__())
gtp.getTPUInfo()
gtp.strategy

In [ ]:
bs="""
    def getTPUInfo(self):
        from tensorflow.python.profiler import profiler_client
        tpu_profile_service_address = os.environ['COLAB_TPU_ADDR'].replace('8470', '8466')
        print(profiler_client.monitor(tpu_profile_service_address, 100, 2))
        tpu_profile_service_address = os.environ['COLAB_TPU_ADDR'].replace('8466', '8470')


getTPUInfo()
"""
print(bs)

In [ ]:
q

In [ ]:
import tensorflow as tf
import timeit


with strategy.scope():
    random_image_cpu = tf.random.normal((1000, 1000, 1000, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    tf.math.reduce_sum(net_cpu)

bs="""
def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))
"""

In [ ]:
q

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()

if device_name != '/device:GPU:0':
    print('GPU device not found')
else: 
    print('Found GPU at: {}'.format(device_name))

In [ ]:
import tensorflow as tf
print(tf.config.list_physical_devices())
if len(tf.config.list_physical_devices()) == 1:
    print('using cpu')
    
    strategy = tf.distribute.get_strategy()
    with strategy.scope():
        print(strategy)
elif len(tf.config.list_physical_devices()) == 2:
    print('using gpu')
    
    strategy = tf.distribute.MirroredStrategy()
    print(strategy)
else:
    print('tpu')


In [ ]:
import tensorflow as tf
if tf.config.list_physical_devices('GPU'):
    print(tf.config.list_physical_devices(), '\n')
    strategy = tf.distribute.MirroredStrategy()

elif tf.config.list_physical_devices('TPU'):
    print(tf.config.list_physical_devices(), '\n')
    strategy = tf.distribute.TPUStrategy()

elif tf.config.list_physical_devices('CPU'):
    print(tf.config.list_physical_devices(), '\n')
    strategy = tf.distribute.get_strategy()

with strategy.scope():
  # Do something interesting
    print(tf.Variable(1.))

In [ ]:
dir(tf.test)

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name() # this will tell you device number (should be 0 with a single GPU)
tf.test.gpu_device_name()       # this will tell you device number (should be 0 with a single GPU)

In [ ]:
import tensorflow as tf
device = tf.config.get_visible_devices()
print(len(device))
print(device)
for dev in device:
    print(dev)
# dir(tf.config)

In [ ]:
import tensorflow as tf
try:
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(resolver)
    tf.tpu.experimental.initialize_tpu_system(resolver)
    print("All devices: ", tf.config.list_logical_devices('TPU'))
    strategy = tf.distribute.TPUStrategy(resolver)
except ValueError:
    strategy = tf.distribute.get_strategy()
    print(strategy)

In [ ]:
dir(tf.distribute)

In [ ]:
import tensorflow as tf
from tensorflow.python.platform.tf_logging import log
logicalDevices = tf.config.list_logical_devices()
print(type(logicalDevices))
print(len(logicalDevices))
for dev in logicalDevices:
    print(dev)

In [ ]:
bs="""
import tensorflow as tf
try:
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(resolver)
    tf.tpu.experimental.initialize_tpu_system(resolver)
    print("All devices: ", tf.config.list_logical_devices('TPU'))
    strategy = tf.distribute.TPUStrategy(resolver)
except ValueError:
    strategy = tf.distribute.get_strategy()
"""
print(bs)